# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

This is a classification problem because the output is discreet. Either stdents need early intervention or they don't. Regression problems deal with continuous outputs (ie real numbers) and the output for this problem definitely is not continuous.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
from IPython.display import display

In [3]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns[:-1])
n_passed = len(student_data[student_data.passed == "yes"])
n_failed = len(student_data[student_data.passed == "no"])
grad_rate = 100. * n_passed / n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [5]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"

pd.options.display.max_columns = None
display(X_all[:15].T)  # print the first 15 rows (as a transpose table)

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
school,GP,GP,GP,GP,GP,GP,GP,GP,GP,GP,GP,GP,GP,GP,GP
sex,F,F,F,F,F,M,M,F,M,M,F,F,M,M,M
age,18,17,15,15,16,16,16,17,15,15,15,15,15,15,15
address,U,U,U,U,U,U,U,U,U,U,U,U,U,U,U
famsize,GT3,GT3,LE3,GT3,GT3,LE3,LE3,GT3,LE3,GT3,GT3,GT3,LE3,GT3,GT3
Pstatus,A,T,T,T,T,T,T,A,A,T,T,T,T,T,A
Medu,4,1,1,4,3,4,2,4,3,3,4,2,4,4,2
Fedu,4,1,1,2,3,3,2,4,2,4,4,1,4,3,2
Mjob,at_home,at_home,at_home,health,other,services,other,other,services,other,teacher,services,health,teacher,other
Fjob,teacher,other,other,services,other,other,other,teacher,other,other,health,other,services,other,other


### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [6]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [7]:
from sklearn.cross_validation import train_test_split

# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=47)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [8]:
# Train a model
import time

def train_classifier(clf, X_train, y_train, print_output=True):
    if print_output:
        print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    training_time = end - start
    if print_output:
        print "Done!\nTraining time (secs): {:.3f}".format(training_time)
    return training_time

# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import SVC
clf = SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.006


0.00642704963684082

In [9]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target, print_output=True):
    if print_output:
        print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    prediction_time = end - start
    if print_output:
        print "Done!\nPrediction time (secs): {:.3f}".format(prediction_time)
    return (prediction_time, f1_score(target.values, y_pred, pos_label='yes'))

train_prediction_time, train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.860927152318


In [10]:
# Predict on test data
test_prediction_time, test_f1_score = predict_labels(clf, X_test, y_test)
print "F1 score for test set: {}".format(test_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.828025477707


In [11]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test, print_output=True):
    if print_output:
        print "------------------------------------------"
        print "Training set size: {}".format(len(X_train))
    training_time = train_classifier(clf, X_train, y_train, print_output)
    train_prediction_time, train_f1_score = predict_labels(clf, X_train, y_train, print_output)
    test_prediction_time, test_f1_score = predict_labels(clf, X_test, y_test, print_output)
    if print_output:
        print "F1 score for training set: {}".format(train_f1_score)
        print "F1 score for test set: {}".format(test_f1_score)
    return (training_time, train_prediction_time, train_f1_score, test_prediction_time, test_f1_score)

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [12]:
# TODO: Train and predict using two other models

### Initial Model Investigation and Comparative Analysis

So far, we know the following about the student intervention classification. We know that the target for the classification model is a single binary class (either a student has passed or they have not). We also know that all of the input features are categorical. Even a student's age can be treated as categorical given that the values are integers and the range of ages for students is reasonably small. Given the large set of categorical features, it is unlikely the data set contains a linear decision boundary between the two target classes. For this reason we will not include any linear classifiers in this exploration.

We do not have a prior model for what makes a student pass or not. For this reason non-parametric models are valid candidates for the Student Intervention Classifier. In general non-parametric models make fewer assumptions about the population from which a sample is taken [1].

We will also consider the potential of Ensemble Learning Models and that one of them may be the most appropriate classifier. The ensemble method uses multiple algorithms in an attempt to improve model performance. Even though multiple algorithms may improve performance, that performance comes at a cost (primarily training and prediction time) [2].

In an attempt to be rigorous in this investigation, it feels prudent to investigate more than 3 classification algorithms especially as the cost of adding additional algorithms to the process is relatively low. For this reason and given the details for this classification problem discussed above, we will investigate the following algorithms: K Nearest Neighbors, SVM, Random Forest, AdaBoost, and Naive Bayes. These 5 algorithms include include two non-parametric models, two ensemble learning models, and Naive Bayes is a relatively simple probabilistic model. We will use Naive Bayes as a benchmark as it is well understood. We will now compare the general applications, strengths, weaknesses, justifications and general performance of these algorithms using their default configurations (as implemented in the SKLearn library). The performance will be calculated with the first 100, first 200 and all 300 training records. Our hope is that from this investigation there will be one algorithm from the 5 that will present itself as the most appropriate choice as the Student Intervention Classifier.

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# For the sake of consistency and as there is little reason not to, this investigation will consider the 
# classifiers using their default settings.

classifiers = [
        {"name": "Nearest Neighbors",
            "clf": KNeighborsClassifier(), 
            "general_applications": "Primarily used for classification when the similarity between neighboring " \
                 "data is relevant and when a meaningful measurement of distance can be defined [3].",
            "strengths": "East to understand. Fast to train. [3]",
            "weaknesses": "Need to determine K. Need to decide how distance is measured and defined. "           \
                "Sensitive to local structure in the data. The data is required for predictions. [3]",
            "justification": "Non-parametric model. See explanation above."},
        {"name": "Support Vector Machine",
            "clf": SVC(), 
            "general_applications": "Typically used for problems with a target variable of  2 classes with a "   \
                 "well defined decision boundary [4].",
            "strengths": "Maximisation of generalisation ability. No local minima. Robustness to outliers. [4]",
            "weaknesses": "Slow to train. Have to decide on kernel and parameter selection. [4]",
            "justification": "Non-parametric model. See explanation above."},
        {"name": "Random Forest",
            "clf": RandomForestClassifier(), 
            "general_applications": "A robust predictive model especially suitable for problems where "          \
                 "overfitting is a concern [5].",
            "strengths": "The use fo multiple trees results in increased predictive power and reduces the "      \
                 "prediction variance [6].",
            "weaknesses": "The use fo multiple trees results in the loss interpretability of a single "          \
                 "decision tree [6]. Need to decide how many trees and their max depth.",
            "justification": "Ensemble method. See explanation above."},
        {"name": "AdaBoost",
            "clf": AdaBoostClassifier(), 
            "general_applications": "AdaBoost is 'often referred to as the best out-of-the-box classifier' [7] " \
                 "and can be applied to a wide variety of classification problems.",
            "strengths": "Boosting can be seen as from of L1 regularization which helps prevent overfitting "    \
                 "and eliminating 'irrelevant' features[6]. Maximises the margin on the training set [8].",
            "weaknesses": "Sensitive to noisy data and outliers [7].",
            "justification": "Ensemble method. See explanation above."},
        {"name": "Naive Bayes",
            "clf": GaussianNB(), 
            "general_applications": "A classic use of Naive Bayse is for text classification and "               \
                 "categorisation [9].",
            "strengths": "Fast to train. Easy to mix and match features of different types [10].",
            "weaknesses": "Can suffer from overfitting [11]. The run-time cost may be too high for some "        \
                 "applications [11].",
            "justification": "Reasonably simple probabalistic model. Will act as a benchmark for the other models."}
    ]

df_columns = ["Name", "General.Applications", "Strengths", "Weaknesses", "Justification", "Train.Set.Size", 
              "Train.Time", "Train.Pred.Time", "Train.Pred.F1", "Test.Pred.Time", "Test.Pred.F1"]
df_indices = df_columns[:6]
classifier_investigation_df = pd.DataFrame(columns=df_columns)

for classifier in classifiers:
    row = {col:0 for col in df_columns}
    row["Name"] = classifier["name"]
    row["General.Applications"] = classifier["general_applications"]
    row["Strengths"] = classifier["strengths"]
    row["Weaknesses"] = classifier["weaknesses"]
    row["Justification"] = classifier["justification"]
    clf = classifier["clf"]
    for n in [100, 200, 300]:
        row["Train.Set.Size"] = n
        training_times_and_scores = train_predict(clf, X_train[:n], y_train[:n], X_test, y_test, False)
        row["Train.Time"] = training_times_and_scores[0]
        row["Train.Pred.Time"] = training_times_and_scores[1]
        row["Train.Pred.F1"] = training_times_and_scores[2]
        row["Test.Pred.Time"] = training_times_and_scores[3]
        row["Test.Pred.F1"] = training_times_and_scores[4]
        
        df_length = len(classifier_investigation_df)
        classifier_investigation_df.loc[df_length] = row
        
indexed_df = classifier_investigation_df.set_index(df_indices)
indexed_df.T

Name                                                                                                                                                             Nearest Neighbors  \
General.Applications  Primarily used for classification when the similarity between neighboring data is relevant and when a meaningful measurement of distance can be defined [3].   
Strengths                                                                                                                                   East to understand. Fast to train. [3]   
Weaknesses           Need to determine K. Need to decide how distance is measured and defined. Sensitive to local structure in the data. The data is required for predictions. [3]   
Justification                                                                                                                         Non-parametric model. See explanation above.   
Train.Set.Size                                                                                                                                                                 100   
Train.Time                                                     0.000725                                                                                                              
Train.Pred.Time                                                0.001669                                                                                                              
Train.Pred.F1                                                  0.867925                                                                                                              
Test.Pred.Time                                                 0.001261                                                                                                              
Test.Pred.F1                                                   0.818182                                                                                                              

Name                                      \
General.Applications                       
Strengths                                  
Weaknesses                                 
Justification                              
Train.Set.Size             200       300   
Train.Time            0.000511  0.000690   
Train.Pred.Time       0.003145  0.005144   
Train.Pred.F1         0.852349  0.846682   
Test.Pred.Time        0.001690  0.002573   
Test.Pred.F1          0.777778  0.837838   

Name                                                                                                      Support Vector Machine  \
General.Applications Typically used for problems with a target variable of  2 classes with a well defined decision boundary [4].   
Strengths                                   Maximisation of generalisation ability. No local minima. Robustness to outliers. [4]   
Weaknesses                                                  Slow to train. Have to decide on kernel and parameter selection. [4]   
Justification                                                                       Non-parametric model. See explanation above.   
Train.Set.Size                                                                                                               100   
Train.Time                                                     0.001074                                                            
Train.Pred.Time                                                0.000646                                                            
Train.Pred.F1                                                  0.858896                                                            
Test.Pred.Time                                                 0.000612                                                            
Test.Pred.F1                                                   0.834356                                                            

Name                                      \
General.Applica

### Extrema for each of the 5 performance metrics

In [14]:
list_of_rows = classifier_investigation_df.T.to_dict().values()
for field in ["Train.Time", "Train.Pred.Time", "Train.Pred.F1", "Test.Pred.Time", "Test.Pred.F1"]:
    print field
    min_val = classifier_investigation_df[field].min()
    min_row = [r for r in list_of_rows if r[field] == min_val][0]
    print "Minimum: " + str(min_val) + " (" + min_row["Name"] + " | " + str(min_row["Train.Set.Size"]) + ")"
    max_val = classifier_investigation_df[field].max()
    max_row = [r for r in list_of_rows if r[field] == max_val][0]
    print "Maximum: " + str(max_val) + " (" + max_row["Name"] + " | " + str(max_row["Train.Set.Size"]) + ")"
    print "\n"

Train.Time
Minimum: 0.000510931015015 (Nearest Neighbors | 200.0)
Maximum: 0.10049700737 (AdaBoost | 100.0)


Train.Pred.Time
Minimum: 0.000236988067627 (Naive Bayes | 100.0)
Maximum: 0.00514388084412 (Nearest Neighbors | 300.0)


Train.Pred.F1
Minimum: 0.622641509434 (Naive Bayes | 100.0)
Maximum: 0.992907801418 (AdaBoost | 100.0)


Test.Pred.Time
Minimum: 0.000236988067627 (Naive Bayes | 100.0)
Maximum: 0.0040168762207 (AdaBoost | 200.0)


Test.Pred.F1
Minimum: 0.415841584158 (Naive Bayes | 100.0)
Maximum: 0.837837837838 (Nearest Neighbors | 300.0)




## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

### A Memo to the Board of Supervisors

#### We Should Apply K-Nearest Neighbors as the Student Intervention Classifier

From the results gathered comparing K-Nearest Neighbors, Support Vector Machine, Random Forests, AdaBoost, and Naive Bayes classifiers we have found that the K-Nearest Neighbors offers the greatest potential as the Student Intervention Classifier. Of those 5 algorithms, KNN is very close to the lowest training time and has the highest test prediction F1 score. The F1 score takes a value from 0 to 1 where 1 is better than 0. Models with high precion and recall, or in other words high proportional true positives and true negatives, have higher F1 scores [13]. Even though KNN has a higher prediction time than some of the other algorithms, it still feels appropriate to select it. The low training time will allow hyperparameter optimisation via grid search to be executed faster allowing us to test a greater number of parameters without having to compromise on the cost to do so. Fundamentally, the high F1 score is one of the greatest benefits of choosing KNN.

As well as the benefits offered by KNN with respect to performance and cost, KNN is also a non-parametric model. Non-parametric models make fewer assumptions about the population from which the training sample is taken (when compared to parametric models) [1]. Given that we do not have a prior model for what makes a student pass of not, the use of a non-parametric model is ideal. It follows the basic intuition about the Student Intervention problem that students are likely to share the classification with other students with whom they are most similar. In addition to all of the eveidence in favour of KNN, KNN reflects this same intuition when determining how to predict the classification of a new datum.

#### How K-Nearest Neighbors Helps us Classify Students

When the KNN algorithm receives a request to predict the classification of a new datum, it searches the set of available data with known classifications. From this search of the labeled data, a subset of K data are selected that are deemed to be the most similar to the unclassified datum. A single classification is then derived from the data in this subset and that classification is returned as the predicted class for the unclassified datum. At face value, this process is very straight forward. What makes the process a little tricky is deciding how we determine the following parameters: how the search is performed, what is the value of K, how is similarity determined, and how is a single classification derived from the K most similar data. For each of these unknowns there is a finite set of options. The easiest way to find the optimal configuration is what is erferred to as hyperparameter optimisation. In effect, every combination of the available parameter options are tested and the combination that yields the best performing algorithm is chosen [14]. Once we have an optimised configuration for KNN, we can calculate the F1 score against the test data and see how well we can expect the algorithm to generalise the Student Inetrvention Classification for new students.

#### Run Hyperparameter Optimisation on the KNN Classifier

In [15]:
# TODO: Fine-tune your model and report the best F1 score

from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn import grid_search
from sklearn.metrics import make_scorer

clf = KNeighborsClassifier()
# The 4 parameters that are discussed above: K, the search algorithm, how similarity is calculated, and how a 
# single classification is derived.
search_parameters = {
    'n_neighbors': range(1, 30), 
    'algorithm': ['auto', 'ball_tree', 'brute'],
    # A selection of distance metrics that are conceptually consistent for categorical data [15][16][17] along with 
    # euclidean and manhattan. We can use distances like euclidean and manhattan because the categorical  
    # variables have been converted to dummy variables, not including them results in a higher test F1 score.    
    'metric': ['euclidean', 'manhattan', 'hamming', 'jaccard', 'dice', 'kulsinski'],
    'weights': ['uniform' , 'distance']
}

scorer = make_scorer(f1_score, pos_label='yes')
ssscv = StratifiedShuffleSplit( y_train, n_iter=10, test_size=0.1)
knn_grid_search = grid_search.GridSearchCV(clf, search_parameters, scoring=scorer, cv=ssscv)
knn_grid_search = knn_grid_search.fit(X_train, y_train)

In [16]:
print "Optimised KNN Model Parameters: " + str(knn_grid_search.best_params_)
print "Optimised KNN Model Training Set F1 Score: " + str(knn_grid_search.best_score_)

Optimised KNN Model Parameters: {'n_neighbors': 19, 'metric': 'manhattan', 'weights': 'uniform', 'algorithm': 'brute'}
Optimised KNN Model Training Set F1 Score: 0.83106817195


#### Optimised KNN Test F1 Score

In [17]:
y_pred = knn_grid_search.best_estimator_.predict(X_test)
test_f1_score = f1_score(y_test, y_pred, pos_label='yes')
print "Optimised KNN Model Test Set F1 Score: " + str(test_f1_score)

Optimised KNN Model Test Set F1 Score: 0.822784810127


In [18]:
# A note from the reviewer
# Pro Tip (Advanced): You could actually go well beyond grid search and implement ‘pipelines’ where the whole machine 
# learning process becomes 'grid-searchable' and you can parameterize and search the whole process though cross 
# validation.
# http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

# And yes you can try out several algorithms automatically as well too! Watch out though this is pretty 
# advanced stuff, here is a great, informative, top notch tutorial from Zac Sewart!
# http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

### References:

[1] Wikipedia contributors, "Nonparametric statistics," Wikipedia, The Free Encyclopedia, https://en.wikipedia.org/w/index.php?title=Nonparametric_statistics&oldid=704406561 (accessed March 6, 2016).

[2] Wikipedia contributors, "Ensemble learning," Wikipedia, The Free Encyclopedia, https://en.wikipedia.org/w/index.php?title=Ensemble_learning&oldid=707462154 (accessed March 6, 2016).

[3] Wikipedia contributors, "K-nearest neighbors algorithm," Wikipedia, The Free Encyclopedia, https://en.wikipedia.org/w/index.php?title=K-nearest_neighbors_algorithm&oldid=708512163 (accessed March 6, 2016).

[4] Abe, Shigeo. "1 Introduction." In *Support Vector Machines for Pattern Classification*, 3. London: Springer, 2005.

[5] Abe, Shigeo. "2.5 Two Class Support Vector Machines - Advantages and Disadvantages." In *Support Vector Machines for Pattern Classification*, 39. London: Springer, 2005.

[6] Breiman, Leo. "Random Forests." *Machine Learning* 45, no. 1 (2001): 5-32. doi:10.1023/a:1010933404324.

[7] Murphy, Kevin P. "16.2.5 Classification and Regression Trees (CART) - Random Forests." In *Machine Learning A Probabilistic Perspective*, 552. Cambridge, Mass: MIT Press, 2012.

[8] Wikipedia contributors, "AdaBoost," Wikipedia, The Free Encyclopedia, https://en.wikipedia.org/w/index.php?title=AdaBoost&oldid=707326699 (accessed March 6, 2016).

[9] Murphy, Kevin P. "16.4.8 Adaptive basis function models - Why does boosting work so well?" In *Machine Learning A Probabilistic Perspective*, 564. Cambridge, Mass: MIT Press, 2012.

[10] Wikipedia contributors, "Naive Bayes classifier," Wikipedia, The Free Encyclopedia, https://en.wikipedia.org/w/index.php?title=Naive_Bayes_classifier&oldid=707015212 (accessed March 6, 2016).

[11] Murphy, Kevin P. "3.5 Naive Bayes classifiers." In *Machine Learning A Probabilistic Perspective*, 84. Cambridge, Mass: MIT Press, 2012.

[12] Murphy, Kevin P. "3.5.4 Naive Bayes classifiers - Feature selection using mutual information." In *Machine Learning A Probabilistic Perspective*, 89. Cambridge, Mass: MIT Press, 2012.

[13] Wikipedia contributors, "F1 score," Wikipedia, The Free Encyclopedia, https://en.wikipedia.org/w/index.php?title=F1_score&oldid=703225135 (accessed March 6, 2016).

[14] Wikipedia contributors, "Hyperparameter optimization," Wikipedia, The Free Encyclopedia, https://en.wikipedia.org/w/index.php?title=Hyperparameter_optimization&oldid=705078252 (accessed March 6, 2016).

[15] Rajaraman, Anand, and Jeffrey D. Ullman. "3.5.3 Jaccard Distance." In *Mining of Massive Datasets*, 94. New York, N.Y.: Cambridge University Press, 2012.

[16] Rajaraman, Anand, and Jeffrey D. Ullman. "3.5.6 Hamming Distance." In *Mining of Massive Datasets*, 96. New York, N.Y.: Cambridge University Press, 2012.

[17] Wikipedia contributors, "Qualitative variation," Wikipedia, The Free Encyclopedia, https://en.wikipedia.org/w/index.php?title=Qualitative_variation&oldid=705908713 (accessed March 6, 2016).